In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
file_path = 'merged_data/final_data.csv'


In [3]:
data = pd.read_csv(file_path)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0.1           9994 non-null   int64  
 1   Country_Code           9994 non-null   object 
 2   total_votes            9994 non-null   int64  
 3   average_quality_score  9994 non-null   float64
 4   Unnamed: 0             9994 non-null   int64  
 5   Year                   9994 non-null   int64  
 6   GDP                    9994 non-null   int64  
 7   Population             9994 non-null   float64
 8   Country_Name           9994 non-null   object 
 9   GDP_per_Capital        9994 non-null   float64
 10  population_rank        9994 non-null   float64
 11  gdp_rank               9994 non-null   float64
 12  gdp_per_capita_rank    9994 non-null   float64
 13  total_votes_rank       9994 non-null   float64
 14  average_quality_rank   9994 non-null   float64
 15  stro